In [1]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import numpy as np
import string
import operator
import csv
import urllib2
import json
from collections import defaultdict

In [2]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "trec7n8"
method = "lr" # hal, mi, assoc, assocMi, assocHal, assoc2
simMeasure = "" # mi, cnet
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfg infile Name =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfg outfile Name =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("collection index Dir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
methodGraphsFileName = []
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method.translate(None, string.digits) in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method.translate(None, string.digits), knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
countsResultsFile = os.path.join(projectDir,"occuranceCount","results",collection+".txt")
print("countsResultsFile =", countsResultsFile)
conceptnet5RelAllFilename ="/scratch/saeid/data/conceptnet5_simp.csv"
print("conceptnet5RelAllFilename =", conceptnet5RelAllFilename)
#expansionCount = 25

cfg infile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/queryTrec7n8
cfg outfile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg
collection index Dir = /scratch/index/indri_5_7/trec7n8
knowledgGraph index Dir = /scratch/index/indri_5_7/trec7n8
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/trec7n8/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/lr/trec7n8/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/lr/trec7n8/cnet/indriRunQuery.evals
countsResultsFile = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/trec7n8.txt
conceptnet5RelAllFilename = /scratch/saeid/data/conceptnet5_simp.csv


In [3]:
conceptnet5RelAll = defaultdict(dict)
conceptnet5RelAllInv = defaultdict(dict)
with open(conceptnet5RelAllFilename, 'r') as f:
    reader = csv.reader(f, delimiter = ",")
    for line in list(reader):
        if line[0].strip() != "" and line[1].strip() != "" and line[2].strip() != "":
            conceptnet5RelAll[line[1].strip()][line[2].strip()] = line[0].strip()
            conceptnet5RelAllInv[line[2].strip()][line[1].strip()] = line[0].strip()
print("size of conceptnet5RelAll =", len(conceptnet5RelAll))
for i, (k1, k2v) in enumerate(conceptnet5RelAll.iteritems()):
    for j, (k2, v) in enumerate(k2v.iteritems()):
        if (i>3 or j>3):
            break
        print (k1, '\t', k2, '\t', v)
        

size of conceptnet5RelAll = 3207471
tripolitan 	 tripoline 	 Synonym
tripolitan 	 tripoli 	 RelatedTo
age of nemesis 	 band 	 IsA
age of nemesis 	 organisation 	 InstanceOf
age of nemesis 	 progressive rock 	 dbpedia/genre
age of nemesis 	 progressive metal 	 dbpedia/genre
joseph john annabring 	 person 	 InstanceOf
british rail class 438 	 mean of transportation 	 InstanceOf
british rail class 438 	 train 	 InstanceOf


In [4]:
count_history = dict()
with open(countsResultsFile, 'r') as f:
    reader = csv.reader(f, delimiter = "\t")
    count_history = {k:int(float(v)) for k,v in list(reader)}
print("size of count_history =", len(count_history))
for i, (k, v) in enumerate(count_history.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

size of count_history = 81446
#uw(#4( storm ) #4( be appear in sky )) 	 0
#uw(#4( work on sundai )) 	 137
#uw(#4( calm time )) 	 23
#uw(#4( support ) #4( comput troubleshoot )) 	 0
#uw(#4( kill ) #4( prai )) 	 394
#uw(#4( bank ) #4( teller offic )) 	 5
#uw(#4( water ) #4( hydrogen )) 	 347
#uw(#4( drug ) #4( take )) 	 8847
#uw(#4( space ) #4( mostli empti )) 	 3
#uw(#4( chang ) #4( edg )) 	 3803
#uw(#4( happin )) 	 0
#uw(#4( trade ) #4( countri )) 	 45838
#uw(#4( game ) #4( leisur activ )) 	 23
#uw(#4( round in shape )) 	 4
#uw(#4( good ) #4( other person )) 	 411
#uw(#4( common eat disord )) 	 1
#uw(#4( spice up salad )) 	 1
#uw(#4( financi depositori )) 	 0
#uw(#4( good ) #4( hear nice new )) 	 0
#uw(#4( teacher ) #4( teach student of class )) 	 0
#uw(#4( bank ) #4( teller monei )) 	 23
#uw(#4( swap share )) 	 106


In [5]:
def splitStemText(text):
    stemmedWords = []
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    if method in {"assocRestful", "lr"}:
        for text_ in text.split(' '):
            text = text_.replace(" ", "_")
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            text = os.path.basename(data_j[u'uri'])
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            #print("data_j =", data_j)
            #print("data_j[u'uri'] =", data_j[u'uri'])
            stemmedWords += [str(os.path.basename(data_j[u'uri']))]
        #print("stemmedWords =", stemmedWords)
        return (stemmedWords)
    else:
        words = text.split()
        for w in words:
            w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
            w = re.sub('\'','', w) # remove apostrophe
            cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
            #print("cmd =", cmd)
            stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
        return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [6]:
rWords1 = dict()
rWords = dict()
if(len(methodGraphsFileName)>0):
    if method != "dir":
        print(methodGraphsFileName[0])
        print("reading graphs...")
        with open(methodGraphsFileName[0], 'r') as graphFile:
            oWord = ""
            for i, row in enumerate(graphFile):
                columns = row.split('\t')
                if(len(columns)==1):
                    oWord = columns[0].strip()
                else:
                    rWord = columns[0].strip()
                    meas = np.float(columns[1].strip())
                    if oWord in rWords:
                        rWords[oWord] += [{rWord:meas}]
    #                        rWords1[oWord][rWord] = meas
                    else:
                        rWords[oWord] = [{rWord:meas}]
                        #rWords1[oWord] = {rWord:meas}
    print("size of rWords =", len(rWords))

In [7]:
if(len(methodGraphsFileName)>1):
    print(methodGraphsFileName[1])
    
    if method != "dir":
        print("reading graphs...")
        with open(methodGraphsFileName[1], 'r') as graphFile:
            oWord = ""
            for i, row in enumerate(graphFile):
                columns = row.split('\t')
                if(len(columns)==1):
                    oWord = columns[0].strip()
                else:
                    rWord = columns[0].strip()
                    meas = np.float(columns[1].strip())
                    if oWord in rWords1:
#                        rWords1[oWord] += [{rWord:meas}]
                        rWords1[oWord][rWord] = meas
                    else:
#                        rWords1[oWord] = [{rWord:meas}]
                        rWords1[oWord] = {rWord:meas}
    print("size of rWords1 =", len(rWords1))

In [8]:
#rWords1 = dict()
#import operator
#print(rWords1["consum"])
#print(sorted(rWords1["consum"], key=operator.itemgetter(1)))

In [9]:
for j, (ow, rws) in  enumerate(rWords.iteritems()):
    print (ow)
    for i, rw in enumerate(rws):
        print ("--> ", rw)
        if i>10:
            break
    if j>20:
        break

In [10]:
cmd = ' '.join(["dumpindex", colIndexDir, "s", "|", "awk", "\'NR==2{print $2}\'"])
c_ = subprocess.Popen(cmd , shell=True, stdout=subprocess.PIPE).stdout.read()
#print(c_)
N = int(c_)

def mi_(N_w, N_v, N_wv):
    #N_w = count_1(w )
    #N_v = count_1(v )
    #N_wv = count_2(w, v)
    pw1 = N_w / float(N)
    pw0 = 1 - pw1
    pv1 = N_v / float(N)
    pv0 = 1 - pv1
    
    pw1v1 = N_wv / float(N)
    pw1v0 = (N_w - N_wv) / float(N)
    pw0v1 = (N_v - N_wv) / float(N)
    pw0v0 = 1 - pw0v1 - pw1v0 - pw1v1
    
    #print("w, v =", w, v)
    #print("N_w, N_v, N_wv, N =", N_w, N_v, N_wv, N)
    #print("pw0 = %f, pw1 = %f, pv0 = %f, pv1 = %f"%(pw0, pw1, pv0, pv1))
    #print("pw0v0 = %f, pw0v1 = %f, pw1v0 = %f, pw1v1 = %f"%(pw0v0, pw0v1, pw1v0, pw1v1))
    
    if N_w == 0 or N_v == 0 or N_wv == 0:
        return 0
    if pv0 == 0 or pw0 == 0 or pv1 == 0 or pw1 == 0:
        return 0
    if pv1 > 0.1 or pw1 > 0.1:
        return 0

    mi = pw1v1*np.log(pw1v1/(pw1*pv1)) + pw1v0*np.log(pw1v0/(pw1*pv0)) + pw0v1*np.log(pw0v1/(pw0*pv1)) + pw0v0*np.log(pw0v0/(pw0*pv0))
    return mi

In [11]:
def countExpr(count_history, origWords, rWords):
    expressions = set()
    for ow in origWords:
        rWords_ow = rWords.get(ow, [])
        if ow.translate(None, string.punctuation).strip() == "":
            continue
        #print (ow)
        expr_ow = "#uw(#4( " + ow.translate(None, string.punctuation) + " ))"
        if(expr_ow not in count_history):
            expressions.add(expr_ow)
        for rw_wv in rWords_ow:
            (rw_w, rw_v) = rw_wv.items()[0]
            #print(rw_w)
            if rw_w.translate(None, string.punctuation).strip() == "":
                continue
            expr_rw = "#uw(#4( " + rw_w.translate(None, string.punctuation) + " ))"
            if(expr_rw not in count_history):
                expressions.add(expr_rw)
            expr_ow_rw = "#uw(#4( " + ow.translate(None, string.punctuation) + " ) #4( " + rw_w.translate(None, string.punctuation) + " ))"
            if(expr_ow_rw not in count_history):
                expressions.add(expr_ow_rw)
    statement = ""
    for expr in expressions:
        statement += expr + "\n"
    with open(os.path.join(projectDir,"occuranceCount","statement.txt"), 'w') as f:
        f.write(statement)
    cmd = ' '.join([os.path.join(projectDir,"occuranceCount","occuranceCount"), colIndexDir, 'x', os.path.join(projectDir,"occuranceCount","statement.txt")])
    #print("cmd =", cmd)
    cAll = subprocess.Popen(cmd , shell=True, stdout=subprocess.PIPE).stdout.read().split("\n")
    countExpr_ = dict()
    for line in cAll:
        line = line.split(':')
        line = [l.strip() for l in line]
        #print("line =", line)
        if len(line) == 2:
            countExpr_[line[0]] = np.float(line[1])
    count_history = dict(count_history.items() + countExpr_.items()) 
    #print(countExpr_)
    return count_history

In [12]:
def countExpr_get_1(count_history, w):
    if w.translate(None, string.punctuation).strip() == "":
        return 0;
    #print (w)
    return count_history["#uw(#4( " + w.translate(None, string.punctuation) + " ))"]
def countExpr_get_2(count_history, w, v):
    if w.translate(None, string.punctuation).strip() == "" or v.translate(None, string.punctuation).strip() == "":
        return 0;
    return count_history["#uw(#4( " + w.translate(None, string.punctuation) + " ) #4( " + v.translate(None, string.punctuation) + " ))"]

In [13]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(count_history, rWords, origWords, expansionCount):
    #print("origWords =", origWords)
    text_string = ""
    
        
    relWords = dict()
    if method == "lr":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        for ow in origWords_:
            #relWords_l += list(conceptnet5RelAll[ow])
            print(ow, "---", conceptnet5RelAll[ow])
            for w in conceptnet5RelAll[ow].keys():
                if w in relWords:
                    relWords[w]+=1
                else:
                    relWords[w]=1 
    elif method == "assocRestful":
        #print("origWords =", origWords)
        origWords_ = [w.replace(" ", "_") for w in origWords]
        url = "http://conceptnet5.media.mit.edu/data/5.4/assoc/list/en/"+','.join(origWords_)
        #print("url =", url)
        response = urllib2.urlopen(url)
        data = response.read()
        data_j = json.loads(data)
        #print("data_j =", data_j)
        relWords = {str(d[0].encode('utf-8')).replace('/c/en/', '').replace('-', ' '):np.float(d[1]) for d in data_j[u'similar'] if '/c/en/' in str(d[0].encode('utf-8'))}
        #print("data_j[u'similar'] =", data_j[u'similar'])
        #print("data_similar =", data_similar)
        
    else:
        count_history = countExpr(count_history, origWords, rWords)
        expressions = set()
        for ow in origWords:
            rWords_ow = rWords.get(ow, [])
            expressions.add("#4( " + ow.translate(None, string.punctuation) + " )")
            if ow.strip() == "":
                continue
            if simMeasure == "mi":
                N_w = countExpr_get_1(count_history, ow)
            for rw_wv in rWords_ow:
                (rw_w, rw_v) = rw_wv.items()[0]
                if rw_w.strip() == "":
                    continue           
                if simMeasure == "mi":
                    N_v = countExpr_get_1(count_history, rw_w)
                    N_wv = countExpr_get_2(count_history, ow, rw_w)
                    rw_v_ = mi_(N_w, N_v, N_wv)
                elif simMeasure == "cnet":
                    rw_v_ = rw_v
                relWords[rw_w] = relWords.get(rw_w, 0) + rw_v_
    
    relWords_sorted = sorted(relWords.items(), key=operator.itemgetter(1), reverse=True)
    #print ("relWords_sorted =", relWords_sorted)
    #counter = 0
    relWords_sel = []
    for counter, (rw_w, rw_v) in enumerate(relWords_sorted):
        if (counter >= expansionCount):
            break
        if  all(c in string.printable for c in rw_w):
            if rw_w not in origWords:
                rw_w = regex.sub(' ', rw_w)
                text_string += rw_w + " "
                relWords_sel += [rw_w]
        #counter += 1
    #print("counter =", counter)
    return(count_history, text_string, relWords_sel)

In [14]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(count_history, intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8"}:
            tags = soup.find_all(['top'])
            #print(tags)

        for tag in tags:
            #print("tag =", tag)
            #print('docno =', tag.find('docno').string)
            #print('text =', tag.find('text').string.strip())
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            if collection in {"aquaint", "gov"}:
                docno_tag.string = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno_tag.string = result.group(1).strip()
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(\n" 

            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
                    
            text_tag.string += str(intCoeff0) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ")\n"

            
            count_history, text_string, relWords_sel1 = getRelWords(count_history, rWords, origWords, expansionCount1)
            #print("text_string =", text_string)
            if len(text_string)>3:
                text_tag.string += str(intCoeff1) + " #combine(" 
                #print("text_string =", text_string)
                text_tag.string += text_string.encode('utf-8')
                text_tag.string += ")\n"

            origWords_new = relWords_sel1[0:N1]#[i[0] for i in relWords_sel]
            count_history, text_string, relWords_sel2 = getRelWords(count_history, rWords, origWords_new, expansionCount2)
            #print("text_string =", text_string)
            if len(text_string)>3:
                text_tag.string += str(intCoeff2) + " #combine(" 
                #print("text_string =", text_string)
                text_tag.string += text_string.encode('utf-8')
                text_tag.string += ")\n"
            
            #print("origWords =", origWords)
            #print("relWords_sel1 =", relWords_sel1)
            #print("relWords_sel2 =", relWords_sel2)
            
            
            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")
        outFile.write(soupNewStr)
    return count_history

In [15]:
def precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s):
    mapPrecs = dict()
    for N1 in N1s:
        for intCoeff0 in intCoeffs0:
            for intCoeff1 in intCoeffs1:
                for intCoeff2 in intCoeffs2:
                    for expansionCount1 in expansionCounts1:
                        for expansionCount2 in expansionCounts2:
                            for dirCoeff in dirCoeffs:
                                count_history = genQueries(count_history, intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1)
                                subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                #print ("cmd = ", cmd)
                                mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                #mapPrecs[intCoeff] = mapPrec
    return count_history

In [16]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.1 ]
#intCoeffs1 = np.arange(0, 0.5, 0.05)
expansionCounts1 = [ 1350 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
#N1s = range(1, 30, 1)
#mapPrecs = 
count_history = precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s)


falkland --- {'falklands': 'RelatedTo', 'place': 'InstanceOf', 'build': 'InstanceOf', 'falkland islander': 'RelatedTo', 'falkland island': 'RelatedTo', 'architectural structure': 'InstanceOf'}
petroleum --- {'oil': 'IsA', 'commodity': 'IsA', 'asphalt': 'PartOf', 'unrefined oil': 'IsA', 'petrol': 'Synonym', 'gas': 'Synonym', 'energy': 'IsA', 'fossil fuel': 'IsA', 'gasoline': 'Synonym', 'just store solar energy': 'IsA', 'liquid hydrocarbon mixture': 'IsA', 'combustible fluid': 'IsA', 'big problem': 'IsA', 'carbon': 'MadeOf', 'linoleum': 'RelatedTo', 'hydrocarbon': 'RelatedTo'}
exploration --- {'old': 'RelatedTo', 'gather information': 'IsA', 'clark': 'RelatedTo', 'find new': 'RelatedTo', 'lewis': 'RelatedTo', 'adventure': 'RelatedTo', 'consideration': 'IsA', 'find': 'RelatedTo', 'album': 'InstanceOf', 'style': 'RelatedTo', 'end': 'NotHasProperty', 'wilderness': 'RelatedTo', 'exploratory': 'RelatedTo', 'musical work': 'InstanceOf', 'travel': 'RelatedTo', 'expedition': 'IsA', 'into unknown

In [17]:
print("length of count_history =", len(count_history))
with open(countsResultsFile, 'w') as f:
    for k, v in count_history.iteritems():
        f.write(k + "\t" + str(v) + "\n")

length of count_history = 81446
